In [1]:
from sentence_transformers import SentenceTransformer, util
passage_encoder = SentenceTransformer('facebook-dpr-ctx_encoder-single-nq-base')

query_encoder = SentenceTransformer('facebook-dpr-question_encoder-single-nq-base')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [5]:
import pandas as pd
df = pd.read_json('C:/Users/ngoph/Desktop/zalo_ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl', lines=True)

In [6]:
df.head(5)

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",2022-05-12 12:46:53+00:00,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society\n\nInternet Society hay ISOC ...,2022-01-20 07:59:10+00:00,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt\n\nTiếng Việt, cũng gọi là tiếng Vi...",2022-05-29 03:42:42+00:00,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio\n\nOhio (viết tắt là OH, viết tắt cũ là O...",2022-04-17 08:15:22+00:00,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California\n\nCalifornia (phát âm như ""Ca-li-p...",2022-06-16 15:27:07+00:00,68738039


In [3]:
passages = [
    "London [SEP] London is the capital and largest city of England and the United Kingdom.",
    "Paris [SEP] Paris is the capital and most populous city of France.",
    "Berlin [SEP] Berlin is the capital and largest city of Germany by both area and population."
]
passage_embeddings = passage_encoder.encode(passages)

query = "What is the capital of England?"
query_embedding = query_encoder.encode(query)

#Important: You must use dot-product, not cosine_similarity
scores = util.dot_score(query_embedding, passage_embeddings)
print("Scores:", scores)

Scores: tensor([[77.8886, 65.5388, 64.5764]])
